In [3]:
import numpy as np
import pandas as pd
import yfinance as yf

In [52]:
def price_barrier_option(option_type, ticker, K, barrier_price):
    # Descargar precios históricos
    data = yf.download(ticker, start='2022-01-01', end='2025-03-15', progress=False)['Close']['SPY']
    daily_returns = data.pct_change().dropna()

    # Parámetros
    S0 = float(data.iloc[-1])
    sigma = daily_returns.std() * np.sqrt(252)
    r = 0.10
    T_days = 252
    T = 1  # años
    dt = T / T_days
    num_simulations = 10000

    Z = np.random.normal(0, 1, (num_simulations, T_days))
    returns = (r - 0.5 * sigma**2) * dt  +  sigma * np.sqrt(dt) * Z
    prices = S0 * np.cumprod(1 + returns, axis=1)

In [54]:
price_barrier_option( 'put', 'SPY', 620, 630)